In [2]:
### 버전관리

In [3]:
!nvidia-smi

Fri Nov 28 12:58:40 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:4C:00.0 Off |                  N/A |
| 39%   30C    P8              18W / 350W |      3MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

from pathlib import Path
from transformers import BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datetime import datetime

import wandb
wandb.login(key="")                     # API KEY 입력

# wandb 꺼두기
os.environ["WANDB_DISABLED"] = "true"

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


### Config 정의

In [5]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")

In [66]:
config_data = {
    "general": {
        "data_path": "../data/",
        "data_raw_path": "../data/raw/",                                       
        "model_name": "digit82/kobart-summarization",                           # 모델 이름
        "output_dir": "../outputs/checkpoints/"                                 
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 100,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#',
                           '#PhoneNumber#', '#Address#', '#PassportNumber#']    # 특정 단어들이 분해되지 않도록 하기 위함
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 50,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "none"  
        # "report_to": "wandb"                                                        # wandb 사용 옵션
    },
    "wandb": {
        # "entity": "bubblekid43",                                                   
        "project": "NLP-private",                                                   
        "name": "baseline_v1"                                                       # 버전관리
    },
    "inference": {
        "ckt_path": "../outputs/checkpoints/",                                      
        "result_path": "../outputs/prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size" : 32,
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}",
                          f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]       # 불필요한 생성 토큰 정의
    },
}

In [67]:
# 모델의 구성 정보를 YAML 파일로 저장
config_path = "../configs/config_v1.yaml"                                            # 버전관리
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)
print(f"💾 config 저장 완료 : {config_path}")

# 저장된 config 파일 불러와서 내용 확인
with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)
print("\n🔍 config 구성 정보 확인")
print(loaded_config)

💾 config 저장 완료 : ../configs/config_v1.yaml

🔍 config 구성 정보 확인
{'general': {'data_path': '../data/', 'data_raw_path': '../data/raw/', 'model_name': 'digit82/kobart-summarization', 'output_dir': '../outputs/checkpoints/'}, 'inference': {'batch_size': 32, 'ckt_path': '../outputs/checkpoints/', 'early_stopping': True, 'generate_max_length': 100, 'no_repeat_ngram_size': 2, 'num_beams': 4, 'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'], 'result_path': '../outputs/prediction/'}, 'tokenizer': {'bos_token': '<s>', 'decoder_max_len': 100, 'encoder_max_len': 512, 'eos_token': '</s>', 'special_tokens': ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']}, 'training': {'do_eval': True, 'do_train': True, 'early_stopping_patience': 3, 'early_stopping_threshold': 0.001, 'evaluation_strategy': 'epoch', 'fp16': True, 'generation_max_length': 100, 'gradient_accumulation_steps': 1, 'learning_rate': 1e-05, 'load_best_model_at_end': True, 'logging_dir': './logs', 'l

### 데이터 로드

In [9]:
# 데이터 경로 정의
data_path = Path(loaded_config['general']['data_path'])

In [10]:
# train data
train_df = pd.read_csv(data_path / 'raw' / 'train.csv')
print('✅ train_df.head()')
print(train_df.head())
display(train_df.head())

✅ train_df.head()
     fname                                           dialogue  \
0  train_0  #Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다...   
1  train_1  #Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?\n#Pers...   
2  train_2  #Person1#: 저기요, 열쇠 세트 본 적 있어요?\n#Person2#: 어떤 ...   
3  train_3  #Person1#: 너 여자친구 있는 거 왜 말 안 했어?\n#Person2#: 미...   
4  train_4  #Person1#: 안녕, 오늘 너무 멋져 보이네요. 저랑 춤 한 곡 추실래요?\n...   

                                             summary      topic  
0  Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필...       건강검진  
1  Mrs. Parker가 Ricky와 함께 백신 접종을 위해 방문하였고, Dr. Pe...      백신 접종  
2  #Person1#은 열쇠 세트를 잃어버리고 #Person2#에게 찾는 것을 도와달라...      열쇠 분실  
3  #Person1#은 #Person2#가 여자친구가 있고 결혼할 예정이라는 사실을 말...  여자친구와의 결혼  
4  Malik은 Wen과 Nikki에게 춤을 제안하고, Wen은 발을 밟는 것을 감수하...       춤 제안  


,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다...","Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필...",건강검진
1,train_1,"#Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?\n#Pers...","Mrs. Parker가 Ricky와 함께 백신 접종을 위해 방문하였고, Dr. Pe...",백신 접종
2,train_2,"#Person1#: 저기요, 열쇠 세트 본 적 있어요?\n#Person2#: 어떤 ...",#Person1#은 열쇠 세트를 잃어버리고 #Person2#에게 찾는 것을 도와달라...,열쇠 분실
3,train_3,#Person1#: 너 여자친구 있는 거 왜 말 안 했어?\n#Person2#: 미...,#Person1#은 #Person2#가 여자친구가 있고 결혼할 예정이라는 사실을 말...,여자친구와의 결혼
4,train_4,"#Person1#: 안녕, 오늘 너무 멋져 보이네요. 저랑 춤 한 곡 추실래요?\n...","Malik은 Wen과 Nikki에게 춤을 제안하고, Wen은 발을 밟는 것을 감수하...",춤 제안


In [11]:
# validation data
val_df = pd.read_csv(data_path / 'raw' / 'dev.csv')
print('✅ val_df.head()')
print(val_df.head())
display(val_df.head())

✅ val_df.head()
   fname                                           dialogue  \
0  dev_0  #Person1#: 안녕하세요, 오늘 기분이 어떠세요?\n#Person2#: 요즘 ...   
1  dev_1  #Person1#: 야 Jimmy, 오늘 좀 이따 운동하러 가자.\n#Person2...   
2  dev_2  #Person1#: 나 진짜 건강에 안 좋은 음식 좀 그만 먹어야겠어. \n#Per...   
3  dev_3  #Person1#: 너 UFO 믿어?\n#Person2#: 당연하지, 있는 거 아냐...   
4  dev_4  #Person1#: 오늘 학교 갔어?\n#Person2#: 당연히 갔지. 너는?\n...   

                                             summary      topic  
0  #Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하...      의사 상담  
1   #Person1#는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.      운동 계획  
2  #Person1#은 건강에 안 좋은 음식을 그만 먹기로 결심하고, #Person2#...     건강한 식단  
3  #Person2#는 UFO를 믿고 꿈에서 볼 수 있다고 한다. #Person1#은 ...   UFO와 외계인  
4  #Person1#은 오늘 학교에 가지 않았고, #Person2#는 내일 학교 대신 ...  학교와 주말 계획  


,fname,dialogue,summary,topic
0,dev_0,"#Person1#: 안녕하세요, 오늘 기분이 어떠세요?\n#Person2#: 요즘 ...",#Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하...,의사 상담
1,dev_1,"#Person1#: 야 Jimmy, 오늘 좀 이따 운동하러 가자.\n#Person2...",#Person1#는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.,운동 계획
2,dev_2,#Person1#: 나 진짜 건강에 안 좋은 음식 좀 그만 먹어야겠어. \n#Per...,"#Person1#은 건강에 안 좋은 음식을 그만 먹기로 결심하고, #Person2#...",건강한 식단
3,dev_3,"#Person1#: 너 UFO 믿어?\n#Person2#: 당연하지, 있는 거 아냐...",#Person2#는 UFO를 믿고 꿈에서 볼 수 있다고 한다. #Person1#은 ...,UFO와 외계인
4,dev_4,#Person1#: 오늘 학교 갔어?\n#Person2#: 당연히 갔지. 너는?\n...,"#Person1#은 오늘 학교에 가지 않았고, #Person2#는 내일 학교 대신 ...",학교와 주말 계획


In [12]:
# test data
test_df = pd.read_csv(data_path / 'raw' / 'test.csv')
print('✅ test_df:', test_df.shape)
print(test_df.head())
display(test_df.head())

✅ test_df: (499, 2)
    fname                                           dialogue
0  test_0  #Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요. \n#Per...
1  test_1  #Person1#: 드디어 왔네! 뭐가 이렇게 오래 걸렸어?\n#Person2#: ...
2  test_2  #Person1#: Kate, 여기서 일어난 일을 믿기 힘들 거야.\n#Person...
3  test_3  #Person1#: 생일 축하해, 이거 너를 위한 선물이야, Brian.\n#Per...
4  test_4  #Person1#: 이 올림픽 공원 정말 크다! \n#Person2#: 맞아. 지금...


,fname,dialogue
0,test_0,"#Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요. \n#Per..."
1,test_1,#Person1#: 드디어 왔네! 뭐가 이렇게 오래 걸렸어?\n#Person2#: ...
2,test_2,"#Person1#: Kate, 여기서 일어난 일을 믿기 힘들 거야.\n#Person..."
3,test_3,"#Person1#: 생일 축하해, 이거 너를 위한 선물이야, Brian.\n#Per..."
4,test_4,#Person1#: 이 올림픽 공원 정말 크다! \n#Person2#: 맞아. 지금...


### 데이터 가공 및 데이터셋 클래스 구축
>- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공하기
>- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만들기

In [13]:
# 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성
class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
        ) -> None:

        self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    # 실험에 필요한 컬럼 가져오기
    def make_set_as_df(file_path, is_train = True):
        if is_train:
            df = pd.read_csv(file_path)
            train_df = df[['fname','dialogue','summary']]
            return train_df
        else:
            df = pd.read_csv(file_path)
            test_df = df[['fname','dialogue']]
            return test_df

    # BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행
    def make_input(self, dataset,is_test = False):
        if is_test:
            encoder_input = dataset['dialogue']
            decoder_input = [self.bos_token] * len(dataset['dialogue'])
            return encoder_input.tolist(), list(decoder_input)
        else:
            encoder_input = dataset['dialogue']
            decoder_input = dataset['summary'].apply(lambda x : self.bos_token + str(x))    # Ground truth를 디코더의 input으로 사용하여 학습
            decoder_output = dataset['summary'].apply(lambda x : str(x) + self.eos_token)
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()

In [14]:
# Train에 사용되는 Dataset 클래스를 정의
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}      # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()}     # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        # item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item.update(item2)
        # item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]                                                                  
        item['labels'] = self.labels['input_ids'][idx]
        return item

    def __len__(self):
        return self.len

# Validation에 사용되는 Dataset 클래스를 정의
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}      # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()}     # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        # item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item.update(item2)
        # item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        item['labels'] = self.labels['input_ids'][idx]
        return item

    def __len__(self):
        return self.len

# Test에 사용되는 Dataset 클래스를 정의
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, len):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.len

In [15]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    train_file_path = os.path.join(data_path, 'raw', 'train.csv')
    val_file_path = os.path.join(data_path, 'raw', 'dev.csv')

    # train, validation에 대해 각각 데이터프레임을 구축
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    # print('\n🚀 train, valid 데이터 로드 중...')
    encoder_input_train , decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val , decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)
    print("\n✅ 학습 데이터 로드 완료!")

    # print('\n🚀 train, valid 데이터셋 생성 중...')
    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                            add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_ouputs = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs,len(encoder_input_train))

    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_ouputs = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs, val_tokenized_decoder_ouputs,len(encoder_input_val))
    print("\n✅ 학습 데이터셋 생성 완료!")

    return train_inputs_dataset, val_inputs_dataset

### Trainer 및 Trainingargs 구축하기
>- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의

In [73]:
# 모델 성능에 대한 평가 지표 정의
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성 토큰들을 제거
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']        # config에서 정의한 불필요한 생성 토큰
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('\n🚀 불필요한 생성 토큰 제거 중...')
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    # print('-'*150)
    # print(f"PRED: {replaced_predictions[1]}")
    # print(f"GOLD: {replaced_labels[1]}")
    # print('-'*150)
    # print(f"PRED: {replaced_predictions[2]}")
    # print(f"GOLD: {replaced_labels[2]}")
    print('\n✅ 불필요한 생성 토큰 제거 완료!')

    # 최종적인 ROUGE 점수를 계산
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가
    result = {key: value["f"] for key, value in results.items()}
    return result

In [17]:
# 학습을 위한 trainer 클래스와 매개변수를 정의
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    
    # print('\n🚀 training arguments 정의 중...')
    # set training args
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'],
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],
                learning_rate=config['training']['learning_rate'],
                per_device_train_batch_size=config['training']['per_device_train_batch_size'],
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],
                warmup_ratio=config['training']['warmup_ratio'],
                weight_decay=config['training']['weight_decay'],
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                evaluation_strategy=config['training']['evaluation_strategy'],
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'],
                fp16=config['training']['fp16'],
                load_best_model_at_end=config['training']['load_best_model_at_end'],
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'],
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'],
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                # report_to=config['training']['report_to']                             # wandb 사용 설정
            )

    # wandb 초기화
    # wandb.init(
    #     # entity=config['wandb']['entity'],
    #     project=config['wandb']['project'],
    #     name=config['wandb']['name'],
    # )

    # wandb checkpoints 저장하는 환경 변수를 설정
    # os.environ["WANDB_LOG_MODEL"]="true"
    # os.environ["WANDB_WATCH"]="false"

    # EarlyStopping : Validation loss가 더 이상 개선되지 않을 때 학습을 중단
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('\n✅ training arguments 정의 완료!')
    
    # print('\n🚀 trainer 생성 중...')
    # Trainer 클래스 정의
    trainer = Seq2SeqTrainer(
        model=generate_model,                                                           # 사용자가 사전 학습하기 위해 사용할 모델 입력
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('\n✅ trainer 생성 완료!')

    return trainer

In [18]:
# 학습을 위한 tokenizer와 사전 학습된 모델 불러오기
def load_tokenizer_and_model_for_train(config,device):

    # print('\n🚀 tokenizer & model 로드 중...')
    print(f'\n🤖 모델명 : {config["general"]["model_name"]}')
    model_name = config['general']['model_name']
    bart_config = BartConfig().from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'],config=bart_config)

    special_tokens_dict={'additional_special_tokens':config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model.resize_token_embeddings(len(tokenizer))          # 사전에 special token을 추가했으므로 재구성
    generate_model.to(device)
    # print(generate_model.config)
    # print('✅ tokenizer & model 로드 완료!')

    return generate_model , tokenizer

In [32]:
# 체크포인트 평가 함수
def eval_checkpoint(ckpt_path, config, tokenizer, eval_dataset):
    print(f"\n=== Evaluate {ckpt_path} ===")
    model = BartForConditionalGeneration.from_pretrained(ckpt_path)
    model.resize_token_embeddings(len(tokenizer))

    args = Seq2SeqTrainingArguments(
        output_dir="./tmp_eval",
        per_device_eval_batch_size=config["training"]["per_device_eval_batch_size"],
        predict_with_generate=True,
        do_train=False,
        do_eval=True,
        report_to="none",           # wandb 설정
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        eval_dataset=eval_dataset,
        compute_metrics=lambda pred: compute_metrics(config, tokenizer, pred),
    )

    metrics = trainer.evaluate()
    print(metrics)
    return metrics

In [36]:
# best & 마지막 checkpoint 찾는 함수
def get_best_and_last_checkpoints(config):
    output_dir = Path(config["general"]["output_dir"])
    ckpt_root = Path(config["inference"]["ckt_path"])

    # 마지막 checkpoint 찾기
    ckpt_dirs = [p for p in ckpt_root.glob("checkpoint-*") if p.is_dir()]
    def _get_step(p: Path) -> int:
        return int(p.name.split("-")[-1])
    ckpt_dirs = sorted(ckpt_dirs, key=_get_step)
    last_ckpt = ckpt_dirs[-1]

    # best checkpoint 찾기
    state_path = last_ckpt / "trainer_state.json"
    with open(state_path, "r") as f:
        state = json.load(f)
    best_ckpt = state.get("best_model_checkpoint", None)

    print(f"\n🚀 체크포인트 탐색 중...")
    print("- best checkpoint :", best_ckpt)
    print("- last_ckpt :", last_ckpt)

    return str(best_ckpt), str(last_ckpt)

In [74]:
def extract_exp_config(config: dict) -> dict:
    # 수정 필요
    return {
        "exp_name": config.get("wandb", {}).get("name", "no_name"),
        "model_name": config["general"]["model_name"],
        "num_train_epochs": config["training"]["num_train_epochs"],
        "learning_rate": config["training"]["learning_rate"],
        "per_device_train_batch_size": config["training"]["per_device_train_batch_size"],
        "per_device_eval_batch_size": config["training"]["per_device_eval_batch_size"],
        "warmup_ratio": config["training"]["warmup_ratio"],
        "weight_decay": config["training"]["weight_decay"],
        "gradient_accumulation_steps": config["training"]["gradient_accumulation_steps"],
        "seed": config["training"]["seed"],
        "lr_scheduler_type": config["training"]["lr_scheduler_type"],
        "optim": config["training"]["optim"],
        "generation_max_length": config["training"]["generation_max_length"],
    }

# 한 번의 실험 결과를 exp_log.csv에 누적 저장하는 함수
def log_experiment_result(config: dict, metrics: dict, ckpt_info: dict | None = None,
                          csv_path: str = "../outputs/exp_log.csv",) -> pd.DataFrame:

    row = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        **extract_exp_config(config),
    }

    # 평가지표 붙이기 (eval_* 만 필터링)
    for k, v in metrics.items():
        if k.startswith("eval_"):
            col_name = k.replace("-", "_")
            row[col_name] = v

    # checkpoint 정보가 있으면 추가
    if ckpt_info is not None:
        row.update(ckpt_info)

    # DataFrame으로 변환
    row_df = pd.DataFrame([row])

    # 기존 csv가 있으면 append, 없으면 새로 생성
    csv_path = Path(csv_path)
    if csv_path.exists():
        prev_df = pd.read_csv(csv_path)
        new_df = pd.concat([prev_df, row_df], ignore_index=True)
    else:
        new_df = row_df

    # 저장
    new_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"\n💾 실험 로그 저장 완료: {csv_path}")

    return new_df

### 모델 학습
>- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [ ]:
def main(config):
    # # 사용할 device 정의
    # device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    # print(f'⚡ 사용 디바이스 : {device} | torch 버전 : {torch.__version__}')

    # # 사용할 모델과 tokenizer 불러오기
    # generate_model , tokenizer = load_tokenizer_and_model_for_train(config,device)
    # print('✅ tokenizer 스페셜 토큰 :', tokenizer.special_tokens_map)

    # # 학습에 사용할 데이터셋 불러오기
    # preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token']) # decoder_start_token: str, eos_token: str
    # data_path = config['general']['data_path']
    # train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config,preprocessor, data_path, tokenizer)

    # # Trainer 클래스를 불러오기
    # trainer = load_trainer_for_train(config, generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset)
    # # 저장된 checkpoint 이후부터 실행 (중간에 학습이 중단돼도 문제 없도록 하기 위함)
    # trainer.train()

    # best / last ckpt 자동 찾기
    best_ckpt, last_ckpt = get_best_and_last_checkpoints(config)

    # 두 개 체크포인트 평가 → DataFrame 비교
    all_results = []
    for name, ckpt in [("best", best_ckpt), ("last", last_ckpt)]:
        metrics = eval_checkpoint(ckpt, config, tokenizer, val_inputs_dataset)
        all_results.append({
            "type": name,
            "checkpoint": os.path.basename(ckpt),
            **{k: v for k, v in metrics.items() if k.startswith("eval_")},
        })
    df = pd.DataFrame(all_results)
    display(df.round(4))

    # 실험 로그 작성
    exp_df = log_experiment_result(
        config,
        metrics,
        ckpt_info={"which": "best", "ckpt": best_ckpt},
        csv_path="../outputs/exp_log.csv",
    )

    # wandb 종료
    # wandb.finish()

In [84]:
if __name__ == "__main__":
    main(loaded_config)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.



🚀 체크포인트 탐색 중...
- best checkpoint : ../outputs/checkpoints/checkpoint-1750
- last_ckpt : ../outputs/checkpoints/checkpoint-2250

=== Evaluate ../outputs/checkpoints/checkpoint-1750 ===


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.



🚀 불필요한 생성 토큰 제거 중...
PRED:   #Person1#은 감기에 걸렸지만 천식 검사를 위해 폐 전문의 
GOLD: #Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하고, 천식 검사를 위해 폐 전문의에게 가볼 것을 권합니다.                                                              

✅ 불필요한 생성 토큰 제거 완료!
{'eval_loss': 0.5997087955474854, 'eval_rouge-1': 0.16493341416565194, 'eval_rouge-2': 0.05019222142976799, 'eval_rouge-l': 0.16075778271732963, 'eval_runtime': 5.0832, 'eval_samples_per_second': 98.166, 'eval_steps_per_second': 3.148}

=== Evaluate ../outputs/checkpoints/checkpoint-2250 ===


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.



🚀 불필요한 생성 토큰 제거 중...
PRED:   #Person1#은 감기에 걸렸고,  #Person2# 
GOLD: #Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하고, 천식 검사를 위해 폐 전문의에게 가볼 것을 권합니다.                                                              

✅ 불필요한 생성 토큰 제거 완료!
{'eval_loss': 0.5969681143760681, 'eval_rouge-1': 0.15465663635945834, 'eval_rouge-2': 0.042542628412776784, 'eval_rouge-l': 0.14892862902645612, 'eval_runtime': 5.0999, 'eval_samples_per_second': 97.845, 'eval_steps_per_second': 3.137}


,type,checkpoint,eval_loss,eval_rouge-1,eval_rouge-2,eval_rouge-l,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,best,checkpoint-1750,0.5997,0.1649,0.0502,0.1608,5.0832,98.166,3.148
1,last,checkpoint-2250,0.5970,0.1547,0.0425,0.1489,5.0999,97.845,3.137



💾 실험 로그 저장 완료: ../outputs/exp_log.csv


### 모델 추론
>- test data를 사용하여 모델의 성능을 확인

In [62]:
# tokenization 과정까지 진행된 test 데이터 출력
def prepare_test_dataset(config,preprocessor, tokenizer):
    
    test_file_path = os.path.join(config['general']['data_path'], 'raw', 'test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path,is_train=False)
    test_id = test_data['fname']
    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data,is_test=True)
    print('\n✅ test 데이터 로드 완료!')

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('\n✅ test 데이터셋 생성 완료!')

    return test_data, test_encoder_inputs_dataset

In [63]:
# 추론을 위한 tokenizer와 학습시킨 모델 불러오기
def load_tokenizer_and_model_for_test(config,device):
    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('\n🤖 모델명 :', model_name)

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    # 자동 탐색된 best checkpoint를 모델로 불러오기
    best_ckpt, last_ckpt = get_best_and_last_checkpoints(config)
    generate_model = BartForConditionalGeneration.from_pretrained(best_ckpt)
    generate_model.resize_token_embeddings(len(tokenizer))
    generate_model.to(device)
    print('🧠 최종 모델 :', best_ckpt)
    print('✅ tokenizer & model 로드 완료!')

    return generate_model , tokenizer

In [96]:
# 학습된 모델이 생성한 요약문의 출력 결과 보기
def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print(f'⚡ 사용 디바이스 : {device} | torch 버전 : {torch.__version__}')
    
    generate_model , tokenizer = load_tokenizer_and_model_for_test(config,device)

    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config,preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    # 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰 제거
    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    save_name = "output_baseline_v1.csv"                                # 버전관리
    output.to_csv(os.path.join(result_path, save_name), index=False)         
    print(f"💾 추론 결과 저장 완료 : {result_path}{save_name}")

    print("\n💬 test문 요약 결과")
    print(output)

    return output

In [97]:
# 추론 실행
if __name__ == "__main__":
    output = inference(loaded_config)

⚡ 사용 디바이스 : cuda:0 | torch 버전 : 2.1.0

🤖 모델명 : digit82/kobart-summarization


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.



🚀 체크포인트 탐색 중...
- best checkpoint : ../outputs/checkpoints/checkpoint-1750
- last_ckpt : ../outputs/checkpoints/checkpoint-2250
🧠 최종 모델 : ../outputs/checkpoints/checkpoint-1750
✅ tokenizer & model 로드 완료!

✅ test 데이터 로드 완료!

✅ test 데이터셋 생성 완료!


100%|██████████| 16/16 [00:22<00:00,  1.42s/it]

💾 추론 결과 저장 완료 : ../outputs/prediction/output_baseline_v1.csv

💬 test문 요약 결과
        fname                                            summary
0      test_0   #Person1# 은 Ms. Dawson에게 사내 메모를 작성하고 배포해달라고 요...
1      test_1    Carrefour 교차로 근처에서 교통체증이 계속되었고, #Person1# 은 ...
2      test_2   #Person1# 은 Kate에게 Masha와 Hero가 두 달 동안 별거 중이며...
3      test_3    Brian은 #Person1# 에게 자신의 생일을 축하하기 위해 파티에 왔다고 ...
4      test_4   #Person1# 과 #Person2# 는 올림픽 공원의 크기에 대해 이야기합니다...
..        ...                                                ...
494  test_495    Jack은 Charlie에게 새로운 비디오 게임을 제안하고, 그녀는 숙제를 먼저...
495  test_496   #Person2# 는 #Person1# 에게 시골 음악에 관심을 갖게 된 계기와 ...
496  test_497    Alice는 세탁기에 비누가 들어 있지 않아 불편함을 겪고 있습니다. #Pers...
497  test_498    Steve는 Matthew에게 임대를 갱신하고 싶지 않다고 말하며, Mrs. T...
498  test_499    Frank는 Betsy에게 승진 소식을 전하며, 파티에 150명이 참석할 것이라...

[499 rows x 2 columns]


In [ ]:
# 제출 전 확인
print("🔎 결측치 및 데이터타입 확인")
print("- shape:", output.shape)
print("\n- null check:\n", output.isnull().sum())
print("\n- dtypes:\n", output.dtypes)
print("\n- duplicated id:", output['fname'].duplicated().sum())

print("\n🔎요약 길이 이상치 확인")
output_summary_length = output.copy()
output_summary_length['summary_length'] = output_summary_length['summary'].str.len()
print(output_summary_length['summary_length'].describe())

🔎 결측치 및 데이터타입 확인
- shape: (499, 2)

- null check:
 fname      0
summary    0
dtype: int64

- dtypes:
 fname      object
summary    object
dtype: object

- duplicated id: 0

🔎요약 길이 이상치 확인
count    499.000000
mean     105.683367
std       19.371654
min       66.000000
25%       91.000000
50%      103.000000
75%      117.000000
max      186.000000
Name: summary_length, dtype: float64


In [2]:
# 제출 후 LB 점수 > 실험 로그에 기록
exp_log = pd.read_csv("../outputs/exp_log.csv")
print("📝 실험 로그")
display(exp_log)

📝 실험 로그


,timestamp,exp_name,model_name,num_train_epochs,learning_rate,per_device_train_batch_size,per_device_eval_batch_size,warmup_ratio,weight_decay,gradient_accumulation_steps,...,eval_rouge_l,eval_runtime,eval_samples_per_second,eval_steps_per_second,which,ckpt,lb_rouge_1,lb_rouge_2,lb_rouge_l,lb_final
0,2025-11-28 18:01:14,baseline_v1,digit82/kobart-summarization,20,0.00001,50,32,0.1,0.01,1,...,0.148929,5.0999,97.845,3.137,best,../outputs/checkpoints/checkpoint-1750,0.5634,0.3668,0.4819,47.0683


In [4]:
# 점수만 보기
display(exp_log[["timestamp", "exp_name", "eval_loss", "eval_rouge_1", "eval_rouge_2", "eval_rouge_l", "lb_rouge_1", "lb_rouge_2", "lb_rouge_l", "lb_final"]])

,timestamp,exp_name,eval_loss,eval_rouge_1,eval_rouge_2,eval_rouge_l,lb_rouge_1,lb_rouge_2,lb_rouge_l,lb_final
0,2025-11-28 18:01:14,baseline_v1,0.596968,0.154657,0.042543,0.148929,0.5634,0.3668,0.4819,47.0683


In [5]:
# 전처리
# 데이터 증강, hard case 분석, error pattern에 따른 rule 추가

In [6]:
# Solar + baseline 조합
#     (1) 에러 케이스에 한해 Solar로 후처리
#     (2) baseline 요약이 너무 짧을 때만 Solar 재요약